In [75]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [76]:
#Loading the dataset

path_df = "Latest-News-Classifier-master/0. Latest News Classifier/03. Feature Engineering/Pickles/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [77]:
df

,Unnamed: 0,File_Name,Content,Category,Complete_Filename,id,News_length
0,0,"WASHINGTON, DC - NOVEMBER 24: U.S. first lady ...",Brazil contains about 60% of the Amazon rainfo...,Amazon,NaN,1,606
1,1,"The ""lungs of the planet"" are burning.\n\nAs t...",As thousands of fiery infernos rage across the...,Amazon,NaN,1,599
2,2,"michael barbaro\n\nFrom The New York Times, I’...","This is “The Daily.” Today: More than 26,000 f...",Amazon,NaN,1,533
3,3,"In honor of Amazon Prime Day, let’s take a loo...",Amazon’s flywheel approach means that innovati...,Amazon,NaN,1,583
4,4,Amazon announced in its earnings report for th...,Amazon announced in its earnings report for th...,Amazon,NaN,1,838
...,...,...,...,...,...,...,...
1859,1859,507.txt,Big guns ease through in San Jose\n\nTop-seede...,sport,507.txt-sport,1,1370
1860,1860,508.txt,Almagro continues Spanish surge\n\nUnseeded Ni...,sport,508.txt-sport,1,775
1861,1861,509.txt,Melzer shocks Agassi in San Jose\n\nSecond see...,sport,509.txt-sport,1,1148
1862,1862,510.txt,Mirza makes Indian tennis history\n\nTeenager ...,sport,510.txt-sport,1,1773


In [78]:
df.loc[0]['Content']

'Brazil contains about 60% of the Amazon rainforest, whose degradation could have severe consequences for global climate and rainfall.\n(AP Photo/Victor R. Caivano)CNN —Brazil’s Amazon rainforest is in flames, burning at the highest rate since 2013, when that nation’s space research center first began tracking fires there.\nHow big is the Amazon rainforest and what does it matter?\nThis means more than one-and-a-half soccer fields of Amazon rainforest are being destroyed every minute of every day, INPE has stated.\nBolsonaro recently stated an alternative explanation for the recent spate of Amazon fires.'

## TEXT CLEANING

In [79]:
# Removing the special characters

# \r and \n
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [80]:
df.loc[0]['Content_Parsed_1']

'Brazil contains about 60% of the Amazon rainforest, whose degradation could have severe consequences for global climate and rainfall. (AP Photo/Victor R. Caivano)CNN —Brazil’s Amazon rainforest is in flames, burning at the highest rate since 2013, when that nation’s space research center first began tracking fires there. How big is the Amazon rainforest and what does it matter? This means more than one-and-a-half soccer fields of Amazon rainforest are being destroyed every minute of every day, INPE has stated. Bolsonaro recently stated an alternative explanation for the recent spate of Amazon fires.'

In [81]:
#removing quotes, punctutaion etc

df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

#convert the text tolower case
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

df.loc[0]['Content_Parsed_2'] 

'brazil contains about 60% of the amazon rainforest, whose degradation could have severe consequences for global climate and rainfall. (ap photo/victor r. caivano)cnn —brazil’s amazon rainforest is in flames, burning at the highest rate since 2013, when that nation’s space research center first began tracking fires there. how big is the amazon rainforest and what does it matter? this means more than one-and-a-half soccer fields of amazon rainforest are being destroyed every minute of every day, inpe has stated. bolsonaro recently stated an alternative explanation for the recent spate of amazon fires.'

In [82]:
#punctuation
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


df.loc[0]['Content_Parsed_3']

<ipython-input-82-855789754928>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


'brazil contains about 60% of the amazon rainforest whose degradation could have severe consequences for global climate and rainfall (ap photo/victor r caivano)cnn —brazil’s amazon rainforest is in flames burning at the highest rate since 2013 when that nation’s space research center first began tracking fires there how big is the amazon rainforest and what does it matter this means more than one-and-a-half soccer fields of amazon rainforest are being destroyed every minute of every day inpe has stated bolsonaro recently stated an alternative explanation for the recent spate of amazon fires'

In [83]:
# removing the 's
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

df.loc[0]['Content_Parsed_4']

'brazil contains about 60% of the amazon rainforest whose degradation could have severe consequences for global climate and rainfall (ap photo/victor r caivano)cnn —brazil’s amazon rainforest is in flames burning at the highest rate since 2013 when that nation’s space research center first began tracking fires there how big is the amazon rainforest and what does it matter this means more than one-and-a-half soccer fields of amazon rainforest are being destroyed every minute of every day inpe has stated bolsonaro recently stated an alternative explanation for the recent spate of amazon fires'

## Stemming and Lemmatization

In [84]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [85]:
#df = df.dropna().reset_index(drop = False)

In [86]:
df.loc[0]['Content_Parsed_4']

'brazil contains about 60% of the amazon rainforest whose degradation could have severe consequences for global climate and rainfall (ap photo/victor r caivano)cnn —brazil’s amazon rainforest is in flames burning at the highest rate since 2013 when that nation’s space research center first began tracking fires there how big is the amazon rainforest and what does it matter this means more than one-and-a-half soccer fields of amazon rainforest are being destroyed every minute of every day inpe has stated bolsonaro recently stated an alternative explanation for the recent spate of amazon fires'

In [87]:
#iterating every word for lemmatization

nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [88]:
df['Content_Parsed_5'] = lemmatized_text_list

## STOP WORDS

In [89]:
stop_words = list(stopwords.words('english'))

In [90]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [91]:
# EXAMPLE

example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [92]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-92-3b7196a1b53b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [93]:
#Special character removal o/p
df.loc[0]['Content']

'Brazil contains about 60% of the Amazon rainforest, whose degradation could have severe consequences for global climate and rainfall.\n(AP Photo/Victor R. Caivano)CNN —Brazil’s Amazon rainforest is in flames, burning at the highest rate since 2013, when that nation’s space research center first began tracking fires there.\nHow big is the Amazon rainforest and what does it matter?\nThis means more than one-and-a-half soccer fields of Amazon rainforest are being destroyed every minute of every day, INPE has stated.\nBolsonaro recently stated an alternative explanation for the recent spate of Amazon fires.'

In [94]:
#lower case o/p
df.loc[1]['Content_Parsed_1']

"As thousands of fiery infernos rage across the Amazon rainforest, tropical vegetation, trees, and the fauna they house are being razed. Since August 15, more than 9,500 new forest fires have started across Brazil, primarily in the Amazon basin. (NOAA)'The sky randomly turned dark'As the world's largest rainforest, the Amazon plays a crucial role in keeping our planet's carbon-dioxide levels in check. Farmers and loggers also purposefully set fire to the rainforest to clear swaths of the Amazon for industrial or agricultural use. The Amazon shrunk by 519 square miles (1,345 square kilometers)."

In [95]:
#punctuation signs
df.loc[1]['Content_Parsed_3']

"as thousands of fiery infernos rage across the amazon rainforest tropical vegetation trees and the fauna they house are being razed since august 15 more than 9500 new forest fires have started across brazil primarily in the amazon basin (noaa)'the sky randomly turned dark'as the world's largest rainforest the amazon plays a crucial role in keeping our planet's carbon-dioxide levels in check farmers and loggers also purposefully set fire to the rainforest to clear swaths of the amazon for industrial or agricultural use the amazon shrunk by 519 square miles (1345 square kilometers)"

In [96]:
# 's removal o/p
df.loc[1]['Content_Parsed_4']

"as thousands of fiery infernos rage across the amazon rainforest tropical vegetation trees and the fauna they house are being razed since august 15 more than 9500 new forest fires have started across brazil primarily in the amazon basin (noaa)'the sky randomly turned dark'as the world largest rainforest the amazon plays a crucial role in keeping our planet carbon-dioxide levels in check farmers and loggers also purposefully set fire to the rainforest to clear swaths of the amazon for industrial or agricultural use the amazon shrunk by 519 square miles (1345 square kilometers)"

In [97]:
#stemming and lemmatization o/p
df.loc[1]['Content_Parsed_5']

"as thousands of fiery infernos rage across the amazon rainforest tropical vegetation tree and the fauna they house be be raze since august 15 more than 9500 new forest fire have start across brazil primarily in the amazon basin (noaa)'the sky randomly turn dark'as the world largest rainforest the amazon play a crucial role in keep our planet carbon-dioxide level in check farmers and loggers also purposefully set fire to the rainforest to clear swaths of the amazon for industrial or agricultural use the amazon shrink by 519 square miles (1345 square kilometers)"

In [98]:
#stop words
df.loc[1]['Content_Parsed_6']

" thousands  fiery infernos rage across  amazon rainforest tropical vegetation tree   fauna  house   raze since august 15   9500 new forest fire  start across brazil primarily   amazon basin (noaa)' sky randomly turn dark'  world largest rainforest  amazon play  crucial role  keep  planet carbon-dioxide level  check farmers  loggers also purposefully set fire   rainforest  clear swaths   amazon  industrial  agricultural use  amazon shrink  519 square miles (1345 square kilometers)"

In [99]:
#final o/p
df.head(1)

,Unnamed: 0,File_Name,Content,Category,Complete_Filename,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,0,"WASHINGTON, DC - NOVEMBER 24: U.S. first lady ...",Brazil contains about 60% of the Amazon rainfo...,Amazon,NaN,1,606,Brazil contains about 60% of the Amazon rainfo...,brazil contains about 60% of the amazon rainfo...,brazil contains about 60% of the amazon rainfo...,brazil contains about 60% of the amazon rainfo...,brazil contain about 60% of the amazon rainfor...,brazil contain 60% amazon rainforest whose ...


In [100]:
#removing the unnecesssary columns
df['Complete_Filename'] = df['File_Name'] + '-' + df['Category']

In [101]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [102]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed
0,"WASHINGTON, DC - NOVEMBER 24: U.S. first lady ...",Amazon,"WASHINGTON, DC - NOVEMBER 24: U.S. first lady ...",Brazil contains about 60% of the Amazon rainfo...,brazil contain 60% amazon rainforest whose ...
1,"The ""lungs of the planet"" are burning.\n\nAs t...",Amazon,"The ""lungs of the planet"" are burning.\n\nAs t...",As thousands of fiery infernos rage across the...,thousands fiery infernos rage across amazon...
2,"michael barbaro\n\nFrom The New York Times, I’...",Amazon,"michael barbaro\n\nFrom The New York Times, I’...","This is “The Daily.” Today: More than 26,000 f...",“ daily” today 26000 fire record inside ...
3,"In honor of Amazon Prime Day, let’s take a loo...",Amazon,"In honor of Amazon Prime Day, let’s take a loo...",Amazon’s flywheel approach means that innovati...,amazon’ flywheel approach mean innovation aro...
4,Amazon announced in its earnings report for th...,Amazon,Amazon announced in its earnings report for th...,Amazon announced in its earnings report for th...,amazon announce earn report fourth quarter...


## Coding the labels

In [103]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'Amazon forest': 4
}

In [104]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

### Train - test split

In [105]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

### Text representation

In [106]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [107]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1584, 300)
(280, 300)


### Using the chi square to view the unigrams and bigrams

In [108]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'Amazon forest' category:
  . Most correlated unigrams:
. first
. firm
. find
. film
. years
  . Most correlated bigrams:
. last year
. year old

# 'business' category:
  . Most correlated unigrams:
. bank
. company
. growth
. market
. firm
  . Most correlated bigrams:
. mr blair
. year old

# 'entertainment' category:
  . Most correlated unigrams:
. tv
. music
. award
. star
. film
  . Most correlated bigrams:
. mr blair
. prime minister

# 'politics' category:
  . Most correlated unigrams:
. tory
. blair
. party
. election
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. players
. cup
. champion
. match
. game
  . Most correlated bigrams:
. prime minister
. year old



In [109]:
bigrams

['tell bbc', 'last year', 'mr brown', 'mr blair', 'prime minister', 'year old']

In [110]:
#Saving the files to pickles

In [111]:
df_path_pickle = 'Latest-News-Classifier-master/0. Latest News Classifier/03. Feature Engineering/'


# X_train
with open(df_path_pickle + 'Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open(df_path_pickle + 'Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open(df_path_pickle + 'Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open(df_path_pickle + 'Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open(df_path_pickle + 'Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open(df_path_pickle + 'Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open(df_path_pickle + 'Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open(df_path_pickle + 'Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open(df_path_pickle + 'Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open(df_path_pickle + 'Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)